In [1]:
using MPI
MPI.Init()

MPI.ThreadLevel(2)

In [2]:
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

1

#### send/recv

In [3]:
struct Point
    x::Float64
    y::Float64
end

In [ ]:
p = Point(rank,rank)

if rank == 0
    MPI.send(p,comm; dest=1)
elseif rank>0
    data = MPI.recv(comm; source=(rank-1))
    MPI.send(p,comm; dest = (rank+1)%size)
end
if rank ==0
    data = MPI.recv(comm; source = (size-1))
end

print("MY rank is $(rank)\n I received this: $(data)\n")
MPI.Barrier(comm)

#### Send/Recv!  receiving into buffers

In [ ]:
send_buf = Array{Float64}(undef,10)
recv_buf = Array{Float64}(undef,10)

fill!(send_buf, Float64(rank))

In [ ]:
if rank == 0
    MPI.Send(send_buf,comm; dest=1)
elseif rank>0
    MPI.Recv!(recv_buf, comm; source=(rank-1))
    MPI.Send(send_buf,comm; dest = (rank+1)%size)
end
if rank ==0
    PI.Recv!(recv_buf, comm; source = (size-1))
end

print("MY rank is $(rank)\n I received this: $(recv_buf)\n")
MPI.Barrier(comm)

#### Isend/Irecv!   non-blocking send and receive

In [ ]:
if rank == 0
    send_status = MPI.Isend(send_buf,comm; dest=1)
elseif rank>0
    recv_status = MPI.Irecv!(recv_buf, comm; source=(rank-1))
    send_status = MPI.Isend(send_buf,comm; dest = (rank+1)%size)
end
if rank ==0
    recv_status = MPI.Irecv!(recv_buf, comm; source = (size-1))
end

status = MPI.Waitall!([send_status, recv_status])

print("MY rank is $(rank)\n I received this: $(recv_buf)\n")